# Spark RAPIDS - ArrayExists demo

GPU implementation at [GpuArrayExists](https://github.com/NVIDIA/spark-rapids/blob/4e77f8501ca3ed4927bb2617f208d155b94f17f8/sql-plugin/src/main/scala/com/nvidia/spark/rapids/higherOrderFunctions.scala#L312)

In [ ]:
spark.conf.set('spark.rapids.sql.explain', 'NONE')
spark.conf.set('spark.rapids.sql.enabled', False)
spark

# Test Data

In [ ]:
spark.createDataFrame(
    [
        [ [10]           ],
        [ [10, None]     ],
        [ [20]           ],
        [ [20, None]     ],
        [ [10, 20, None] ],
        [ [None, None]   ],
        [ [],            ], 
        [ None           ],
    ],
    'a array<int>'
).createOrReplaceTempView('df')


In [ ]:
sql("""
    SELECT a,
           transform(a, x -> x = 10) AS trans10 
      FROM df
""").show()

In [ ]:
demo_query="""
    SELECT a,
           exists(a, x -> x = 10)    AS exits10 
      FROM df
"""
spark.conf.set('spark.rapids.sql.enabled', True)

# Spark 3.x default Three Value Logic (3VL)

In [ ]:
spark.conf.set('spark.sql.legacy.followThreeValuedLogicInArrayExists', True)
spark.conf.set('spark.rapids.sql.enabled', True)
sql(demo_query).explain()
sql(demo_query).show()

# Spark 2.x Binary Value Logic (2VL)

In [ ]:
spark.conf.set('spark.sql.legacy.followThreeValuedLogicInArrayExists', False)
sql(demo_query).explain()
sql(demo_query).show()